In [1]:
from dotenv import load_dotenv
from openai import OpenAI
from time import sleep

In [2]:
# load environment variables (API keys)
# should be in .env file OPENAI_API_KEY=sk-...
load_dotenv()

client = OpenAI()


In [ ]:
# Uplaod file should only be done once

file_path = "Larsson.pdf"

file = client.files.create(
    file=open(file_path, "rb"),
    purpose='assistants'
)

print(file.id)

In [ ]:

'''
This is the assistant that will be used to answer questions about the book.

the assistant is only create once and can be used multiple times. with assitant.id
'''
assistant = client.beta.assistants.create(
    name="Islam quiz creator",
    instructions="Create a quiz about Islam from the provided text. Use only the provided text and craete a generall quizz to help people learn about Islam.",
    #
    # retrieval tool is used to implement a RAG-system
    # 
    tools=[{"type": "retrieval"}],
    model="gpt-4-1106-preview",
    file_ids=[file.id]
)

print(assistant.id)

In [6]:

assistantID = "asst_hBggvgQ75jVwE37nchjTVecf"


thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Give me 5 quizz questions about Islam. from the provided text only."
)

run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistantID,
  instructions="Please address the user as Jane Doe. The user has a premium account."
)


while run.status != "completed":
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    sleep(1)

print(run.status)



KeyboardInterrupt: 

In [4]:

# retrieve and format mesasge 
messages = client.beta.threads.messages.list(thread_id=thread.id)
message = messages.data[0]

# Extract the message content
message_content = message.content[0].text
annotations = message_content.annotations
citations = []

# Iterate over the annotations and add footnotes
for index, annotation in enumerate(annotations):
    # Replace the text with a footnote
    message_content.value = message_content.value.replace(
        annotation.text, f' [{index}]')

    # Gather citations based on annotation attributes
    if (file_citation := getattr(annotation, 'file_citation', None)):
        cited_file = client.files.retrieve(file_citation.file_id)
        citations.append(
            f'[{index}] {file_citation.quote} from {cited_file.filename}')
    elif (file_path := getattr(annotation, 'file_path', None)):
        cited_file = client.files.retrieve(file_path.file_id)
        citations.append(
            f'[{index}] Click <here> to download {cited_file.filename}')
        # Note: File download functionality not implemented above for brevity

# Add footnotes to the end of the message before displaying to user
message_content.value += '\n' + '\n'.join(citations)


In [5]:
print(message_content.value)

Based on the provided text, here are five quiz questions about Islam:

1. What year did Prophet Muhammad receive his first revelation and at what location?
   - A) Year 622 in Medina
   - B) Year 610 on Ljusets Berg
   - C) Year 610 in Mecca
   - D) Year 622 on Jabal al-nur
   - Correct answer: B) Year 610 on Ljusets Berg [0].

2. How many chapters (surahs) are there in the Quran?
   - A) 96
   - B) 622
   - C) 114
   - D) 610
   - Correct answer: C) 114 [1].

3. Which of the following is not true according to the Muslim tradition regarding the sequence of the Quran's chapters?
   - A) They are arranged in chronological order of revelation.
   - B) The Quran starts with a short opening chapter called Al-Fatiha.
   - C) The Quran ends with shorter chapters.
   - D) The first revelation is placed as chapter 96.
   - Correct answer: A) They are arranged in chronological order of revelation [1].

4. Which period was mainly characterized by revelations aimed at establishing the early Muslim